In [3]:
from datasets import load_dataset
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from datasets import load_from_disk

# Step 1: Load the dataset
dataset = load_from_disk("legacy-datasets/banking77")

# Step 2: Convert train and test splits to DataFrame
train_data = pd.DataFrame(dataset['train'])
test_data = pd.DataFrame(dataset['test'])

# Step 3: Extract features and labels
X_train = train_data['text']
y_train = train_data['label']
X_test = test_data['text']
y_test = test_data['label']

In [4]:
# Define the intents and their 130 example phrases
intents_examples = {
    "greet": [
        "Hello", "Hi", "Hey", "Good morning", "Good afternoon", "Howdy", "What’s up?", "Hi there", "Yo", "Greetings",
        "How’s it going?", "How do you do?", "What’s happening?", "How are you?", "Hey there", "Good evening", "What’s new?",
        "Hello, how are you?", "Hey, what’s up?", "Hi, how’s everything?", "How’s life?", "Long time no see", "Greetings, how are you?",
        "It’s nice to meet you", "Hey, how’s it going?", "Hi, everyone!", "Morning!", "Sup?", "How’s your day going?", "Hey, you okay?",
        "What’s good?", "How are things?", "Howdy, partner!", "Hello there!", "Welcome!", "Hiya!", "What’s crackin'?", "Hey, what’s new?",
        "Yo, what’s up?", "Good day!", "How’s everything with you?", "Hi, nice to meet you!", "Hey there, how are you?", "What’s popping?",
        "Hello, what's going on?", "Yo, what’s good?", "Hi, what’s new with you?", "Good to see you", "Hey, how’s everything going?",
        "Hello, how’s your day?", "Hi, what’s up with you?", "How’s your week been?", "Morning, how’s it going?", "Hello, how are you today?",
        "Hi, hope you're doing well!", "How’s your morning going?", "What’s up, buddy?", "Hey, nice to talk to you!", "How’s life treating you?",
        "Yo, everything alright?", "Greetings, friend!", "Hey, how’s life been?", "Howdy, how are you today?", "Hello, what’s going on with you?",
        "Hi, how’s everything with you?", "What’s new in your life?", "Hi there, what's good?", "Hello, are you well?", "How's it going, pal?",
        "Yo, long time no see!", "Howdy, what's up?", "Hey, how’s it hanging?", "Hello, everything good?", "Hi, hope you're having a good day!",
        "How are you doing today?", "Hey, how’s everything?", "Hello, how are things?", "Hi, good to see you again!", "What’s happening with you?",
        "How are things on your end?", "Hey, what’s going on today?", "Good to see you!", "How are you feeling today?", "Hey there, what’s up today?",
        "Hi, how’s everything going for you?", "Hello, good to hear from you!", "Hey, how’s life treating you?", "How's everything today?",
        "What’s up with you today?", "Hi, what’s good in your life?", "How’s your day treating you?", "Hey, feeling good today?",
        "How’s it going this morning?", "Hi, how’s your week been going?", "What’s new with you today?", "Greetings, how’s life today?",
        "How’s your day been so far?", "What’s going on in your life?", "Hey, how’s everything on your side?", "Hi, everything okay with you?",
        "What’s good today?", "Hi, all good here?", "How are you today, friend?", "Hello, what’s new in your world?", "How’s everything with you, my friend?",
        "Hey, how are you feeling today?", "Hi, good day ahead!", "How’s the morning treating you?", "Yo, how’s everything?", "Greetings, feeling good?",
        "How’s life for you right now?", "Hey, how’s everything with you today?", "Hello, all good here!", "How’s your life going today?",
        "Hi, what’s up with you right now?", "Morning, how’s it going?", "Howdy, how’s life?", "How’s everything, my friend?", "Yo, how are you today?"
    ],
    "goodbye": [
        "Bye", "Goodbye", "See you later", "Take care", "Have a nice day", "Farewell", "Catch you later", "See ya", "Peace out", "Later",
        "Until next time", "Have a great day", "Talk to you soon", "See you soon", "Goodbye for now", "Take it easy", "Bye for now", "So long",
        "Adieu", "Take care of yourself", "Until we meet again", "Bye-bye", "See you around", "Take care, talk soon", "Have a good one",
        "Alright, see ya", "Catch you on the flip side", "Until next time", "Later, gator", "Take care, bye", "Peace, take care", "Have a good day ahead",
        "Be well", "Catch you later, alligator", "Take care, stay safe", "Goodbye, have a nice day!", "Farewell, stay well!", "Till next time, bye!",
        "I’m outta here", "Bye now", "Take care of yourself", "Have a good rest of your day", "Later, alligator", "Talk soon, bye!", "See you later, alligator!",
        "Have a good night!", "Goodbye, hope to see you again soon", "See you in a bit", "See you soon, take care", "Until we talk again", "Bye for now, take care!",
        "Wishing you a great day", "Take care, see you soon!", "Catch you later, buddy", "Wishing you well, goodbye!", "Bye, take care of yourself!",
        "Goodbye, see you soon!", "Farewell for now", "Catch you on the other side", "Stay safe, bye!", "Take care of business, see you", "See you soon, my friend",
        "See ya, enjoy your day!", "Adios", "I’ll catch you later", "Until we meet again, goodbye", "Have a great one", "Bye, stay happy!", "Be safe, take care",
        "Farewell, talk to you soon!", "Goodbye, hope you enjoy your day!", "Until we meet, take care!", "See you later, take care", "Have a nice one",
        "Talk to you soon, bye!", "Bye, take care of yourself", "Good luck, see you later", "See you tomorrow", "Take care, enjoy your day!", "Until we chat again, goodbye",
        "Bye, be well!", "Have a good rest of the day!", "See you soon, take care!", "Talk to you soon, take care!", "Goodbye, my friend!", "Bye-bye, see you soon!",
        "Farewell, enjoy your day!", "Stay well, see you later", "Goodbye, stay safe!", "Later, talk soon!", "Bye for now, my friend!", "Take care, I’ll see you later",
        "Bye, have a good time!", "Stay well, catch you later", "See you in a bit!", "Catch you later, bye-bye", "Talk soon, take care", "Goodbye, be happy!",
        "See you soon, be safe", "Bye-bye for now!", "Till next time, take care", "Bye, stay safe, see you later", "Goodnight, see you soon!", "Talk soon, take care of yourself",
        "Have a great evening, bye!", "Catch you later, have a good one!", "Stay safe, talk soon", "Take care, have a great time!", "Later, take care of yourself",
        "Take care, have fun!", "Goodbye, stay strong", "See you tomorrow, bye!", "Stay safe, take care", "Take care, see you soon!", "Goodbye, hope you have a good time",
        "Catch you soon, take care!", "See you soon, enjoy your day", "Goodbye, stay well and happy!", "Bye, see you later!"
    ],
    "authenicate": [
         "Login", "Sign in", "Authenticate me", "Please sign me in", "Log into my account", "I want to log in", "Sign me in to my account", 
         "I forgot my password", "I need to authenticate", "Can you help me log in?", "Please authenticate me", "I can't sign in", 
         "How do I log in?", "Log me into my account", "I’m having trouble signing in", "Help me authenticate my account", "Can I log in now?", 
         "Can you authenticate my account?", "I want to access my account", "I need to sign in to my account", "Log me in, please", 
         "I need to log in to check my account", "How do I authenticate my account?", "I’m unable to sign in", "I forgot my username", 
         "How do I recover my password?", "Can I reset my login credentials?", "I need to change my password", "Authenticate me, please", 
         "I forgot my login details", "Can I reset my password now?", "Help me recover my account", "I need a password reset", 
         "Please help me log in again", "My account is locked, help!", "I want to reset my password", "Can you send me a login link?", 
         "Help me log in again", "How can I log into my account?", "I need to access my account again", "I can’t sign in, please help", 
         "Send me a link to sign in", "Can you help me recover my password?", "Reset my password, please", "I lost my password", 
         "Help me recover my password", "I need help with authentication", "Log me into my banking account", "Can you verify my account?", 
         "Send me the login instructions", "Help me with my login issues", "I am unable to log into my account", "What should I do to log in?", 
         "How can I sign in to my account?", "Help me access my account", "How do I access my account again?", "I can’t sign in, can you help?", 
         "Please send me my password recovery options", "I need to authenticate myself", "Can you send me my login credentials?", 
         "I can't remember my password", "How can I access my account again?", "Help me reset my login details", "How can I recover my login details?", 
         "I need a new password", "Please help me log in to my account", "Send me a link to recover my password", "How can I fix my login problem?", 
         "Reset my login credentials", "I’ve forgotten my username", "I need to authenticate with my email", "Log me in using my email address", 
         "Help me sign in with my email", "I need a login link for my account", "Reset my password via email", "Can you send me a verification code to sign in?", 
         "How do I verify my identity to sign in?", "Please help me authenticate my account", "Help me get back into my account", "I’m unable to access my account, help", 
         "I need to reset my password now", "Please verify my identity", "Can you send a verification code to my phone?", "How can I reset my login details?", 
         "I’m locked out of my account", "How do I reset my password?", "Help me with my sign-in issue", "Can you authenticate me with my phone number?", 
         "How can I verify my account now?", "I need a new password for my account", "How can I reset my password on the app?", "I need to change my sign-in details", 
         "I need help with my login", "Can you authenticate me now?", "Help me authenticate my account with a code", "I can’t remember my username", 
         "I need to recover my login credentials", "Reset my account authentication", "Sign in via my phone", "Send me a new password", 
         "I need a code to authenticate my account", "Please log me into my account", "Can you verify my login?", "I can't authenticate, help!", 
         "Help me authenticate using my phone", "How do I verify myself?", "Help me with my login details", "Please help me reset my authentication", 
         "Send me a password reset link", "Log me in using my credentials", "Help me sign in again", "Can you verify me for sign-in?", 
         "I need help authenticating", "Help me recover my sign-in details", "Please authenticate my access", "Can you help me log in again?"
    ],
    "balance_inquiry": [
         "Check my balance", "What’s my account balance?", "How much money do I have?", "Check balance", "Balance inquiry", 
         "What is my balance?", "How much is in my account?", "Please tell me my balance", "Show me my balance", "How much funds do I have?", 
         "Account balance, please", "Can you show me my balance?", "What’s the balance in my account?", "Tell me my account balance", 
         "Can you check my balance?", "How much is in my bank account?", "What’s my remaining balance?", "Show me how much I have", 
         "How much do I have left?", "I need to check my balance", "Check the balance in my account", "What’s left in my account?", 
         "How much do I have right now?", "How much money is in my bank account?", "Give me my account balance", "What is the balance in my account?", 
         "Please check my bank balance", "What’s the amount in my account?", "How much money is left in my account?", "Please tell me how much I have", 
         "Can you check my bank balance?", "Can you tell me my balance?", "How much money is remaining?", "Tell me the balance in my account", 
         "What is the balance of my bank account?", "How much do I have available?", "I want to check my account balance", 
         "Check the available balance in my account", "What’s my current balance?", "Tell me the balance in my account", 
         "Can I see my account balance?", "Can you show me how much I have?", "What is the remaining balance?", "What’s the available balance in my account?", 
         "How much do I have in my checking account?", "Show me how much money is in my account", "What’s my checking account balance?", 
         "Please show me my account funds", "Can you show me my remaining balance?", "How much is in my account right now?", 
         "What is the available balance?", "How much funds do I have left?", "Check my balance right now", "How much is left in my checking account?", 
         "Tell me my available balance", "What’s the total in my account?", "How much do I have in my savings?", "I want to know my balance", 
         "Can you show my available balance?", "What’s the total balance?", "Check my bank account balance", "Can I see how much I have?", 
         "Show me my checking account balance", "What’s the available amount?", "How much is in my savings account?", 
         "What’s the available funds in my account?", "Please tell me the current balance", "I need to know my balance", 
         "Tell me how much I have in total", "How much is available in my account?", "Can you tell me how much is left?", 
         "What’s the total funds in my account?", "How much money is available?", "Can you check my account balance for me?", 
         "What is my account balance at the moment?", "How much funds do I have remaining?", "Can you show me my account balance now?", 
         "What is the current balance?", "Tell me the funds in my account", "What is in my bank account?", "How much do I have in total?", 
         "Can you tell me the balance now?", "How much is available in my checking?", "Please show me my balance now", 
         "Can you check how much I have?", "How much is in my savings account?", "Tell me the balance in my checking", "Can I see my bank balance?"
    ],
    "transfer": [
         "Transfer money", "Send money", "Transfer funds", "I want to send money", "Can you transfer funds for me?", "Send money to someone", 
         "Please transfer the money", "I need to transfer some money", "How do I send money?", "Can I send money now?", "Make a transfer", 
         "How do I transfer funds?", "Send money to my friend", "Please send money to my account", "I need to send funds", "Can you send money now?", 
         "Please transfer some funds", "Send money to my bank account", "Can you make a transfer for me?", "I want to make a transfer", 
         "How do I transfer to another account?", "Please send funds to my bank", "Send money to this account", "How do I send money to another bank?", 
         "I need a transfer to my account", "Please send this amount to my account", "Transfer money to my account", "How can I transfer money to you?", 
         "I need to send money to someone", "Can you help me transfer money?", "Please send funds to my account now", "I want to transfer to another bank", 
         "Send this amount to my account", "How do I send funds?", "Transfer money to my checking account", "Can I transfer funds today?", 
         "Send money to the recipient", "Please transfer the money to my friend", "I want to transfer money to my savings", 
         "How can I send money to my account?", "Transfer this amount to my bank", "I need to transfer money to my bank account", 
         "Please send funds right now", "Transfer funds immediately", "I want to send money quickly", "Can you transfer money to my account?", 
         "Please transfer money to my friend", "I need money transferred", "How can I send money to my account?", "Transfer money between accounts", 
         "How do I send money to someone else?", "Send the funds to my checking", "I need a transfer now", "How can I send money now?", 
         "Please send this amount to my savings", "I want to transfer funds right away", "Can you send this amount for me?", "Please transfer funds now", 
         "Transfer money to my savings account", "I need to send this amount to someone", "How do I send this money?", "Transfer the money to me", 
         "Send the funds now", "How can I send the funds?", "I need to send funds right away", "Please send money to my account", 
         "Can you send money to my friend?", "I want to transfer money quickly", "Can you transfer funds to my savings account?", 
         "Please transfer money to the recipient", "Send funds to my account immediately", "How do I send money to my friend?", "Transfer this amount to me", 
         "Can you make the transfer now?", "Please send money today", "How do I transfer money online?", "Transfer funds to my checking account", 
         "Can I send money right now?", "Please transfer the money to my account", "I want to transfer this amount", "How do I send money to my bank?"
    ],
    "logout": [
         "Log out", "Sign out", "Please log me out", "I want to log out", "Sign me out", "Log me out of my account", 
         "Please sign me out", "Log out now", "Can you log me out?", "I need to log out", "Please sign me out of my account", 
         "Help me log out", "How do I log out?", "Log me out immediately", "Sign me out right now", "Can you sign me out of my account?", 
         "I want to sign out now", "Log me off", "Please log me off", "I want to be logged out", "Can you please log me off?", 
         "I’m done, log me out", "Help me sign out", "Can you please sign me out?", "Please log off", "Sign out of my account now", 
         "Help me log off", "I want to sign off now", "I need to log off my account", "Log me out from my session", "Please log out my account", 
         "Can you help me sign out?", "Sign me off right away", "Log out from my account", "I need to sign out right now", 
         "Log out now, please", "I’m done with this session", "I want to log off my account", "Help me sign off", "I want to end my session", 
         "Sign me off now", "Log me out from here", "Please log out immediately", "Can you sign me off?", "I don’t need this account anymore", 
         "Sign out of this device", "I want to sign out of my account", "Please sign me off", "Help me log out of my account", 
         "I want to close my session", "Can you help me log out of this device?", "Log me off this device", "Please log me out of the app", 
         "Can you sign me off this account?", "Log me out from the app", "I want to log out from here", "Sign out from this account", 
         "Help me log out of the app", "Can I log out now?", "Please end my session", "Log me off now", "Sign out of the application", 
         "Please log me out from this device", "Help me log off the app", "I’m finished, log me off", "Can you sign me out of the system?", 
         "Log me out from this platform", "I want to log off now", "Please help me sign out", "Log out of the website", "I want to log out from the system", 
         "Can you sign me out of the website?", "Please log off this account", "Log me off my account now", "Sign off from this device", 
         "I want to log out from the website", "Log me off the platform", "Please sign off my account", "Help me sign off now", 
         "Can you help me log out of this platform?", "I’m ready to log off", "Log out from my account right now", "Can you end my session?", 
         "Sign me off from here", "Can I please log out now?", "I want to sign out now", "I’m ready to sign off", "Sign off this account", 
         "Please log me off this account", "Can you end my account session?", "Sign out from the platform", "Log me off immediately", 
         "Help me logout now", "Log off my account right away", "Can you log off this account?", "Sign me out from the system", 
         "Log out of my account please", "Sign me off right away", "Can you log out of my account?", "Please log out now", 
         "I’m ready to log out", "I want to log out of this platform", "Log me off the system", "Can you help me sign out of the platform?", 
         "I need to sign out of my account", "Can I sign out right now?", "Log me off right now"
    ],
    "transaction": [
         "Transaction history", "Show me my transaction history", "What are my recent transactions?", "Can you show me my transactions?", 
         "Please show me my transaction details", "I want to see my transaction history", "Can you give me my transaction list?", 
         "What’s my transaction history?", "I need to check my transaction history", "Show me my transaction records", 
         "Can I see my past transactions?", "I want to know my transaction details", "Tell me my recent transactions", 
         "Give me a list of transactions", "What transactions have I made?", "Can I get my transaction history?", "I need my transaction records", 
         "Show me the list of transactions", "Can you check my transaction history?", "I want to review my transactions", 
         "Please tell me my transaction history", "What have I spent recently?", "Show me my spending history", "I want to see my past transactions", 
         "Can you show my transaction details?", "Give me my transaction records", "What transactions were made on my account?", 
         "I need to see my account transactions", "Can you give me the transaction list?", "Please provide me with my transaction details", 
         "How do I view my transaction history?", "Can you show my account history?", "I need to check the transactions in my account", 
         "Please show me my spending details", "Can you provide me with transaction records?", "What transactions have I performed?", 
         "Show me my payments history", "Tell me about my past transactions", "Can you provide me with my transaction summary?", 
         "Give me my recent transaction list", "I want to see my recent payments", "What’s the transaction history on my account?", 
         "How can I see my transaction history?", "Can you list my transactions?", "Please show my payment history", "Show me what I’ve spent", 
         "Give me my account transaction records", "Can I get my transaction details?", "What’s my recent spending?", "Show me my purchases", 
         "Please give me my transaction records", "Can you show me a list of transactions?", "Give me an overview of my transactions", 
         "What’s the history of my account?", "Can you tell me about my past purchases?", "Show me the transactions in my account", 
         "Please show me the history of transactions", "I need to know my transaction list", "What’s the record of my transactions?", 
         "Can you provide my spending history?", "How do I see my spending history?", "Give me my transaction history now", 
         "What’s the breakdown of my transactions?", "Can you provide my recent transactions?", "Tell me about my last transactions", 
         "What have I bought recently?", "Show me what I’ve spent in the last month", "I need to view my past spending", "Can you show me my transaction logs?", 
         "Please give me my past transactions", "What are my recent purchases?", "How do I check my transaction summary?", 
         "Can you list my spending history?", "Show me my past purchases", "Can I review my transactions?", "Give me my transaction report", 
         "Tell me my spending summary", "Show me the details of my transactions", "Please show me a record of my transactions", 
         "What’s the list of my recent transactions?", "I need a summary of my transactions", "Can I review my transaction history?", 
         "Show me my financial transactions", "Can you show my spending details?", "Please provide me with my transaction records", 
         "What purchases have I made?", "Give me a list of my recent transactions", "Can you show me the history of payments?", 
         "What transactions did I perform last week?", "Please list all my transactions", "Tell me what I spent on recently"
    ],
    "loan_status": [
         "Loan status", "What’s the status of my loan?", "Has my loan been approved?", "Can you check my loan status?", 
         "I want to know the status of my loan", "Please tell me about my loan status", "Can you tell me if my loan is sanctioned?", 
         "Is my loan approved?", "What’s the progress of my loan?", "Has my loan been processed?", "Can you give me an update on my loan?", 
         "I want an update on my loan", "Tell me if my loan is sanctioned", "What’s the latest on my loan?", "I need to check my loan status", 
         "Can you provide an update on my loan?", "What is the current status of my loan?", "Has my loan been approved or denied?", 
         "What’s going on with my loan?", "Can you show me the loan status?", "I need to know if my loan was approved", 
         "Tell me about the status of my loan", "Can you check if my loan is approved?", "Is my loan going through?", "Can I get an update on my loan?", 
         "What’s the current loan status?", "Is my loan approved yet?", "Please tell me if my loan is active", "Can you provide loan approval status?", 
         "I need a loan status update", "What’s the status of my loan application?", "How do I check my loan approval?", 
         "What’s the result of my loan application?", "Is my loan processed yet?", "Tell me if my loan is still in progress", 
         "What’s the status of my loan application?", "Is my loan being approved?", "Tell me the status of my loan application", 
         "Can you give me an update on the approval of my loan?", "Please check if my loan is approved", "I want to know if my loan has been processed", 
         "Has my loan been approved or denied?", "Can you show me the loan approval status?", "Is my loan under review?", "When will I know the loan result?", 
         "What’s the approval status of my loan?", "Has my loan been rejected?", "Please update me on my loan status", 
         "Can you check the progress of my loan?", "Is my loan ready?", "Please tell me if my loan is in review", 
         "Has my loan been finalized?", "Can you tell me the loan approval progress?", "I want to track the status of my loan", 
         "How do I know if my loan is approved?", "Check if my loan has been finalized", "Has my loan been sanctioned yet?", 
         "Tell me about the loan approval", "I need to know the loan result", "What’s the decision on my loan?", "Can you provide details about my loan status?", 
         "Is my loan application being reviewed?", "What is the status of my loan approval?", "Has my loan been processed?", 
         "Can you confirm the loan status?", "Please check the status of my loan", "What’s the result of my loan application?", 
         "How do I know if my loan has been sanctioned?", "Can you give me information about my loan?", "I need an update on my loan approval", 
         "What’s happening with my loan?", "Can you check if my loan has been finalized?", "Tell me the latest on my loan", 
         "Can you provide loan approval updates?", "Has my loan application been reviewed?", "What’s the status of my loan now?"
    ],
    "status": [
         "Account status", "What’s the status of my account?", "Is my account active?", "Check my account status", 
         "Can you tell me my account status?", "Is my account locked?", "How’s my account doing?", "What’s my account's status?", 
         "Tell me if my account is working", "Is my account currently active?", "What’s the status of my bank account?", "Can you check if my account is active?", 
         "Is my account functioning properly?", "Can you tell me if my account is locked?", "What’s the state of my account?", 
         "Is my account operational?", "Check if my account is locked", "Tell me if my account is open", "I need to know my account status", 
         "What is the status of my account?", "Is my account in good standing?", "Can you confirm my account is active?", 
         "What’s going on with my account?", "Is my account currently working?", "Tell me if my account is frozen", 
         "Can you confirm if my account is locked?", "What’s the situation with my account?", "Is my account in good standing?", 
         "Can you tell me if my account is active or locked?", "Tell me the status of my account now", "I want to check the account status", 
         "Can you confirm my account is open?", "Is my account working properly?", "What’s happening with my account?", 
         "How do I know if my account is locked?", "Can you tell me if my account is frozen?", "What’s the condition of my account?", 
         "Please check if my account is open", "How can I verify if my account is locked?", "Can you confirm if my account is frozen?", 
         "I need to know the status of my account now", "What is the account condition?", "How do I know if my account is active?", 
         "Check if my account is functioning properly", "What’s the current status of my account?", "Is my account locked or active?", 
         "Can you check if my account is open?", "Is there any issue with my account?", "I want to verify the status of my account", 
         "Can you check the condition of my account?", "Is my account secure?", "Check if my account is working fine", "Is my account fine?", 
         "Can you tell me if my account is working?", "What’s my account's current status?", "Is my account blocked?", 
         "Can you tell me if my account is under review?", "What’s the status of my savings account?", "Can you provide me with an account update?", 
         "What is the current state of my account?", "Is my account operating?", "Can you confirm my account status now?", 
         "Is my account still active?", "Can you show me the status of my current account?", "Tell me the condition of my account", 
         "Is my account in service?", "Can you check if my account is open right now?", "I need to know if my account is still locked", 
         "Is my account working or blocked?", "Can you check if my bank account is locked?", "Tell me about my account status", 
         "Can you help me verify my account's status?", "Is my account available?", "Is my account working today?", "What’s the status of my current balance?", 
         "Can you give me my account’s operational status?", "Tell me if there’s anything wrong with my account", "Is there any restriction on my account?"
    ],
    "bank_add": [
         "Bank address", "What’s the address of the nearest bank?", "Where is the nearest bank?", "Can you tell me the bank’s location?", 
         "Where can I find the nearest ATM?", "Please give me the address of the nearest bank", "Where is the closest branch?", 
         "Can you tell me the nearest bank location?", "Where is the nearest bank branch?", "Find me the nearest bank", 
         "I need the address of the nearest ATM", "Tell me where the nearest bank is", "Find the closest bank branch", 
         "Can you give me the address of the nearest branch?", "Where is the nearest ATM machine?", "Please tell me where the nearest bank is", 
         "Where’s the nearest bank branch?", "Can you show me where the nearest bank is?", "Where can I find a bank?", 
         "Where is the nearest ATM machine?", "Can you find the nearest bank branch?", "Please show me the nearest bank address", 
         "Can you give me the nearest branch address?", "Where is the nearest bank?", "Tell me where to find a bank nearby", "Can you provide the nearest bank’s address?", 
         "Where’s the closest ATM?", "Give me the address of the nearest branch", "Can you find me a bank nearby?", "Where’s the nearest ATM?", 
         "Can you find a bank near me?", "Where’s the closest branch to me?", "Tell me the nearest bank ATM", "I need the closest bank’s address", 
         "Where is the nearest ATM to me?", "Where can I find a nearby ATM?", "Where is the closest bank to my location?", 
         "Where is the nearest bank branch to me?", "I want to find the nearest ATM machine", "Where can I find the closest branch?", 
         "Please give me the address to the nearest ATM", "Where’s the closest bank ATM?", "Where is the nearest ATM located?", 
         "Find the nearest branch to my location", "Can you show me where to find an ATM?", "Where’s the nearest branch to my location?", 
         "Where’s the nearest ATM machine?", "Tell me the closest branch", "Find the closest ATM to me", "Where is the closest bank to me?", 
         "What’s the address of the nearest ATM?", "I need to know where the nearest bank is", "Please give me the location of the nearest branch", 
         "Can you find the nearest ATM?", "Where can I find a nearby branch?", "Where’s the closest ATM nearby?", "Where is the closest bank branch?", 
         "Tell me the nearest ATM address", "Where can I find a bank near me?", "Can you find the nearest ATM branch?", "Where’s the nearest branch to my location?", 
         "Find me the nearest ATM machine", "Can you help me locate the nearest bank?", "What’s the nearest ATM location?", "Where is the nearest ATM?"
    ],
    "info": [
         "Account info", "Can I get my account info?", "Tell me my account details", "What’s my account information?", 
         "Can you show me my account details?", "Give me my account information", "I need my account information", 
         "Show me my account info", "Please provide me with my account details", "I want to know my account info", 
         "Tell me about my account", "Give me information on my account", "Can you check my account details?", 
         "I need to know about my account", "What is my account info?", "Please show me my account information", "What are the details of my account?", 
         "Can I see my account information?", "Can you give me my account info?", "What’s my account data?", 
         "I need to check my account info", "Can I get details of my account?", "Tell me my account details now", "Please give me my account info", 
         "What is the information about my account?", "I want details of my account", "Please give me all the details of my account", 
         "Can you show me my account information now?", "I need to verify my account info", "What’s in my account?", 
         "Give me a summary of my account", "Tell me the details of my account", "Please give me my account details now", 
         "What is my current account information?", "Can you provide me with the details of my account?", "What is the info on my account?", 
         "Can I check my account info?", "What are the details of my bank account?", "Please check my account details", 
         "Give me the details of my account", "Can I have my account details?", "I need to verify the details of my account", 
         "Tell me about my account data", "What’s the information on my account?", "Can you provide details of my bank account?", 
         "Give me my account status", "Can you confirm my account details?", "Please show me the details of my bank account", 
         "Can you display my account information?", "What’s my account status?", "Show me my account data", 
         "Tell me about my bank account", "I need to see my account details", "What’s in my current account?", "Give me a detailed view of my account", 
         "What’s the balance in my account?", "Can you show me the current status of my account?", "Can you give me all my account information?", 
         "I need my account info for verification", "Tell me if there are any restrictions on my account", "What’s the latest information about my account?", 
         "Is my account currently active?", "Can you help me verify my account information?", "What’s the state of my account?", 
         "How do I access my account details?", "Can I view my account info?", "Please show me all the details of my account", 
         "Can you provide me with my account’s current status?", "Tell me about the balance in my account", "I want to check my account’s details", 
         "Can you give me information about my bank account?", "Please give me the latest info on my account", "Where can I check my account details?", 
         "Is there any important information about my account?", "Please confirm the details of my account", "Tell me if there are any updates to my account", 
         "I need to check the details of my savings account", "What’s the account balance?", "Can you display my account history?", 
         "I want to see my account details right now", "Please provide me with my account status and details", "What’s my account summary?", 
         "Give me the details of my savings account", "Can you show me my checking account details?", "What’s my recent account info?", 
         "Is there anything important about my account that I should know?", "Tell me if I need to update my account details", "Can I check my current account info?", 
         "I want to get my latest account details", "Please give me the full details of my bank account", "What’s the history of my account?", 
         "Can you show me all the transactions in my account?", "Tell me about the status of my account now", "Can I review my account details?"
    ],
    "loan_amt": [
         "Loan amount", "What is the loan amount?", "How much do I owe on my loan?", "What’s the remaining balance on my loan?", 
         "How much is left to pay on my loan?", "What is my total loan balance?", "Can you tell me my loan amount?", 
         "How much do I still owe?", "What’s the total amount of my loan?", "How much have I paid on my loan?", 
         "Can you show me the outstanding balance on my loan?", "What’s the remaining loan amount?", "How much is left to be paid?", 
         "Tell me the current balance of my loan", "Can you tell me how much I owe on my loan?", "What is the remaining loan balance?", 
         "How much is the loan balance?", "Please give me the current loan amount", "What’s the amount left on my loan?", 
         "How much do I need to pay to settle my loan?", "What is the principal amount of my loan?", "Can you confirm the remaining loan balance?", 
         "What is the current amount of the loan?", "How much do I have to pay on the loan?", "Tell me how much I still owe on my loan", 
         "What’s the total amount left to pay?", "Can you check my loan amount for me?", "How much is remaining to be paid on my loan?", 
         "How much do I have left to pay on my loan?", "Can you tell me the balance of my loan?", "I need to know the amount left on my loan", 
         "What’s my loan balance?", "How much money do I still owe?", "Please show me my loan balance", "What is the amount I owe on the loan?", 
         "How much have I repaid on the loan?", "Can you give me a breakdown of my loan balance?", "What is the outstanding amount of my loan?", 
         "How much is left to pay on my loan?", "Tell me the remaining amount for my loan", "What’s the balance of my loan now?", 
         "Can you confirm the loan amount for me?", "How much have I paid towards my loan?", "What’s the amount left on my loan to pay off?", 
         "Tell me how much I owe for my loan", "How much is my loan principal?", "What’s the total I owe on the loan?", "Can you give me the exact loan balance?", 
         "What is the current loan remaining?", "How much is left for my loan to be paid off?", "Please confirm my loan balance", 
         "How much do I owe in total for my loan?", "Can you tell me the current loan balance?", "How much is pending on my loan?", 
         "What’s the remaining amount to be paid on my loan?", "What’s my loan balance right now?", "Can you confirm the amount I owe?", 
         "How much of my loan do I still owe?", "What is the remaining payment on my loan?", "Tell me the loan payment balance", 
         "How much do I need to pay off the loan?", "What is the outstanding loan amount?", "How much is remaining on my loan?", 
         "Can you check the remaining amount on my loan?", "Tell me the total amount left on my loan", "What’s the outstanding loan to be paid?", 
         "What is the remaining loan debt?", "How much money is left to pay on my loan?", "Can you show me the loan balance?", 
         "How much is left to be paid on my loan today?", "What’s the balance I need to pay to clear my loan?", "What’s the amount remaining on my loan?", 
         "Can you confirm my loan’s total remaining balance?", "How much do I still need to pay for my loan?", "Tell me the outstanding loan balance", 
         "What’s my remaining loan debt?", "How much is remaining to be paid on my loan?", "Can you tell me the balance left on my loan?", 
         "How much do I owe to settle my loan?", "What is the total remaining balance of my loan?", "Tell me the outstanding amount I owe on my loan"
    ],
    "loan_doc": [
    "What are the documents required for my loan sanction?", "Tell me the documents needed for loan approval", 
    "What papers are necessary for a loan sanction?", "List the required documents for my loan", 
    "Can you tell me the documents needed for loan processing?", "What documents are required for my home loan?", 
    "Give me a list of documents for loan approval", "What paperwork is needed for loan sanction?", 
    "Which documents are necessary for a loan?", "Please tell me the required documents for a loan", 
    "What do I need to submit for my loan approval?", "What documents should I provide for my loan?", 
    "Can you tell me the papers required for a loan?", "What is the checklist for loan sanction documents?", 
    "What files are required to get a loan?", "What documents do I need for my loan processing?", 
    "List all the papers needed for loan approval", "What documents do I have to submit for a loan?", 
    "Can you give me the required documents for loan application?", "What is required for loan sanction?", 
    "Tell me what I need to apply for a loan", "What are the requirements for loan approval?", 
    "Can you list the documents for loan sanction?", "What forms do I need for a loan?", 
    "What supporting documents are required for my loan?", "Tell me the paperwork for loan processing", 
    "What are the documents I need to provide for a loan?", "What are the mandatory documents for loan sanction?", 
    "What documents do I need to get my loan approved?", "What proof is required for a loan?", 
    "What are the document requirements for a loan application?", "Which papers are necessary for a loan?", 
    "Tell me the documents needed for getting a loan approved", "What are the essential documents for loan approval?", 
    "What paperwork do I need for loan processing?", "What documentation is required for a home loan?", 
    "What should I submit to get a loan?", "Can you tell me the documents required to process my loan?", 
    "Which documents do I have to provide for my loan?", "Tell me the documentation needed for loan sanction"
]

}

# Convert the intent examples into a list of dictionaries
augmented_examples = []

for intent, examples in intents_examples.items():
    for phrase in examples:
        augmented_examples.append({"text": phrase, "label": intent})

# Convert the augmented examples into a Dataset
from datasets import Dataset
augmented_dataset = Dataset.from_list(augmented_examples)


In [5]:
from datasets import DatasetDict

total_rows = len(augmented_dataset) 


train_size = len(dataset['train'])  
test_size = len(dataset['test'])    


train_ratio = train_size / (train_size + test_size)
test_ratio = test_size / (train_size + test_size)


train_dataset, test_dataset = augmented_dataset.train_test_split(
    test_size=test_ratio, shuffle=True, seed=42
).values()


augmented_dataset_split = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})


print(augmented_dataset_split)


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 883
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 272
    })
})


In [6]:
from datasets import ClassLabel


original_class_names = dataset['train'].features['label'].names
li = ["greet","goodbye","authenicate","balance_inquiry","transfer","logout","transaction","loan_status","status","bank_add","info","loan_amt"]
for i in li:
    original_class_names.append(i) 


augmented_dataset_split = augmented_dataset_split.map(lambda example: {
    'label': original_class_names.index(example['label']) if example['label'] in original_class_names else -1
}, remove_columns=['label'])


augmented_dataset_split = augmented_dataset_split.cast_column('label', ClassLabel(names=original_class_names))
dataset_split = dataset.cast_column('label', ClassLabel(names=original_class_names))


print(augmented_dataset_split['train'].features)
print(augmented_dataset_split['test'].features)
print(dataset_split['train'].features)
print(dataset_split['test'].features)

Map:   0%|          | 0/883 [00:00<?, ? examples/s]

Map:   0%|          | 0/272 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/883 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/272 [00:00<?, ? examples/s]

{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['activate_my_card', 'age_limit', 'apple_pay_or_google_pay', 'atm_support', 'automatic_top_up', 'balance_not_updated_after_bank_transfer', 'balance_not_updated_after_cheque_or_cash_deposit', 'beneficiary_not_allowed', 'cancel_transfer', 'card_about_to_expire', 'card_acceptance', 'card_arrival', 'card_delivery_estimate', 'card_linking', 'card_not_working', 'card_payment_fee_charged', 'card_payment_not_recognised', 'card_payment_wrong_exchange_rate', 'card_swallowed', 'cash_withdrawal_charge', 'cash_withdrawal_not_recognised', 'change_pin', 'compromised_card', 'contactless_not_working', 'country_support', 'declined_card_payment', 'declined_cash_withdrawal', 'declined_transfer', 'direct_debit_payment_not_recognised', 'disposable_card_limits', 'edit_personal_details', 'exchange_charge', 'exchange_rate', 'exchange_via_app', 'extra_charge_on_statement', 'failed_transfer', 'fiat_currency_support', 'get_disposable_virtual_card'

In [7]:
unique_augmented_labels = set(example['label'] for example in augmented_dataset_split['train'])
print("Unique labels in augmented dataset:", unique_augmented_labels)
print("Original class names:", original_class_names)

Unique labels in augmented dataset: {77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, -1}
Original class names: ['activate_my_card', 'age_limit', 'apple_pay_or_google_pay', 'atm_support', 'automatic_top_up', 'balance_not_updated_after_bank_transfer', 'balance_not_updated_after_cheque_or_cash_deposit', 'beneficiary_not_allowed', 'cancel_transfer', 'card_about_to_expire', 'card_acceptance', 'card_arrival', 'card_delivery_estimate', 'card_linking', 'card_not_working', 'card_payment_fee_charged', 'card_payment_not_recognised', 'card_payment_wrong_exchange_rate', 'card_swallowed', 'cash_withdrawal_charge', 'cash_withdrawal_not_recognised', 'change_pin', 'compromised_card', 'contactless_not_working', 'country_support', 'declined_card_payment', 'declined_cash_withdrawal', 'declined_transfer', 'direct_debit_payment_not_recognised', 'disposable_card_limits', 'edit_personal_details', 'exchange_charge', 'exchange_rate', 'exchange_via_app', 'extra_charge_on_statement', 'failed_transfer', 'fiat_curr

In [8]:
from datasets import concatenate_datasets


dataset['train'] = concatenate_datasets([dataset_split['train'], augmented_dataset_split['train']])


dataset['test'] = concatenate_datasets([dataset_split['test'], augmented_dataset_split['test']])


print(dataset['train'][-44])

{'text': 'I want to know my transaction details', 'label': 83}


In [9]:
dataset.save_to_disk("full_banking77_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/10886 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3352 [00:00<?, ? examples/s]

In [10]:
from datasets import load_from_disk
dataset = load_from_disk("full_banking77_dataset")

In [20]:
print(dataset['train'][-65])

{'text': 'Can you provide my spending history?', 'label': 83}
